In [2]:
import numpy as np
import pandas as pd

In [13]:
# Loading demographic information, then creating income classes
hh_feat = pd.read_csv('hh_demographic.csv')

low_inc = ['Under 15K', '15-24K', '25-34K', '35-49K']
mid_inc = ['50-74K', '75-99K', '100-124K', '125-149K']
up_inc = ['150-174K', '175-199K', '200-249K', '250K+']

def define_income(df):
    if df['INCOME_DESC'] in low_inc:
        return 'Low'
    elif df['INCOME_DESC'] in mid_inc:
        return 'Middle'
    else:
        return 'Upper'
    
hh_feat['family_income'] = hh_feat.apply(lambda x: define_income(x), axis=1)

In [5]:
# Verifying primary descriptions of the most relevant products
data = pd.read_csv('transaction_data.csv')
data = data[data['QUANTITY'] != 0]
data = data[data['SALES_VALUE'] != 0]

products = pd.read_csv('product.csv')

# If only grocery products are desired, then only_groc = 1
only_groc = 0

if only_groc == 1:
    groc_products_only = products[products['DEPARTMENT'] == 'GROCERY']
else:
    groc_products_only = products

data_filtered = data.merge(groc_products_only, on='PRODUCT_ID', how='inner')

data_filtered['prod_price'] = pd.qcut(
    data_filtered['SALES_VALUE'],
    [0, .25, .5, .75, .9, 1],
    labels=False
)

prim_desc_analysis = (
    data_filtered
    .groupby('COMMODITY_DESC')
    .count()
    .sort_values('PRODUCT_ID', ascending=False)
)

sec_desc_analysis = (
    data_filtered
    .groupby('SUB_COMMODITY_DESC')
    .count()
    .sort_values('PRODUCT_ID', ascending=False)
)

In [6]:
# Getting most frequent prod_price from each product
det_price = data_filtered.groupby(['PRODUCT_ID', 'prod_price', 'SALES_VALUE']).count()['DAY'].reset_index()
det_price = det_price.groupby('PRODUCT_ID').first('prod_price')[['prod_price', 'SALES_VALUE']].reset_index()

In [7]:
# Creating product dataframe
groc_products = groc_products_only.merge(det_price, on='PRODUCT_ID') # price information
groc_products['PRODUCT_ID'] = groc_products['PRODUCT_ID'].astype('str')

groc_products = groc_products.rename(
    {'COMMODITY_DESC': 'primary_description',
    'SUB_COMMODITY_DESC': 'secondary_description'}, axis=1
)

groc_products_desc = groc_products[[
    'PRODUCT_ID', 'primary_description', 'secondary_description', 'prod_price',
    'DEPARTMENT', 'SALES_VALUE'
]]

groc_products_desc.to_csv('product_desc.csv', index=False)

In [8]:
# Getting only necessary columns from transactions data
data = data_filtered[['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID']]
data['PRODUCT_ID'] = data['PRODUCT_ID'].astype('str')

# Creating other demographic classes
hh_feat = hh_feat.loc[hh_feat['HH_COMP_DESC'] != 'Unknown']
kids_group = ['1 Adult Kids', '2 Adults Kids']
hh_feat['has_kids'] = np.where(hh_feat['HH_COMP_DESC'].isin(kids_group), 1, 0)
hh_feat = hh_feat.sample(frac=0.2)

/tmp/ipykernel_45329/2674493492.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PRODUCT_ID'] = data['PRODUCT_ID'].astype('str')


In [9]:
def create_interactions(df, feature, demographic_value, file_name):
    # df1: pd1 -> pd2 , user, user_features
    df = df[['household_key', 'DAY', 'PRODUCT_ID', 'BASKET_ID']]

    # Getting timeslice
    df = df[(df['DAY']>365) & (df['DAY']<540)]

    filter_prods = groc_products_desc
    
    # Filtering only interactions with desired products
    df = (
        df
        .merge(filter_prods, on='PRODUCT_ID', how='inner')
    )

    df = df.drop(['prod_price', 'primary_description', 'secondary_description'], axis=1)

    # Creating dataframe with combinations of products bought in the same basket
    df_1 = df.rename(columns={'PRODUCT_ID': 'PRODUCT_ID_1'})
    df = (
        df_1
        .merge(
            df[['PRODUCT_ID', 'BASKET_ID']].rename(columns={'PRODUCT_ID': 'PRODUCT_ID_2'}),
            on='BASKET_ID',
            how='inner'
        )
    )

    # Removing duplicates from the combinatinos of products bought in the same basket
    df['check_string'] = df.apply(
        lambda row: ''.join(sorted([row['PRODUCT_ID_1'], row['PRODUCT_ID_2']])), axis=1
    )
    df = df.drop_duplicates(['BASKET_ID', 'check_string'])
    df = df[df['PRODUCT_ID_1'] != df['PRODUCT_ID_2']]
    prod_interactions = (
        df
        .drop_duplicates(subset=['DAY', 'PRODUCT_ID_1', 'PRODUCT_ID_2'])
    )

    prod_interactions = (
        prod_interactions
        .merge(
            hh_feat[['household_key', feature]],
            on='household_key',
            how='inner'
        )
    )

    # Filtering interactions for desired demographic class
    prod = prod_interactions[prod_interactions[feature]==demographic_value]

    prod.to_csv(file_name, index=False)

In [10]:
# Writing interactions dataframes
create_interactions(data, 'family_income', 'Low', 'interactions_low.csv')
create_interactions(data, 'family_income', 'Middle', 'interactions_middle.csv')
create_interactions(data, 'family_income', 'Upper', 'interactions_upper.csv')

In [11]:
# This function is used to update the product_id column, replacing big numbers
# from the original dataset with the product dataframe index. This is needed so
# that we can use the networks in LargeNetVis
def update_interactions(file_name):
    # Reading interactions
    df = pd.read_csv(file_name).drop(['SALES_VALUE', 'DEPARTMENT'], axis=1) #e.g., interactions_low.csv

    # Updating PRODUCT_ID_1
    prods = pd.read_csv('product_desc.csv')
    prods['PRODUCT_ID_1'] = prods['PRODUCT_ID']
    prods['PRODUCT_ID'] = prods.index
    prods = prods.drop(['prod_price', 'SALES_VALUE', 'primary_description', 'secondary_description', 'DEPARTMENT'], axis=1)

    df = df.merge(prods, on='PRODUCT_ID_1', how='inner')
    df = df.drop('PRODUCT_ID_1', axis=1).rename(columns={'PRODUCT_ID': 'PRODUCT_ID_1'})

    # Updating PRODUCT_ID_2
    prods = pd.read_csv('product_desc.csv')
    prods['PRODUCT_ID_2'] = prods['PRODUCT_ID']
    prods['PRODUCT_ID'] = prods.index
    prods = prods.drop(['prod_price', 'SALES_VALUE', 'primary_description', 'secondary_description', 'DEPARTMENT'], axis=1)

    df = df.merge(prods, on='PRODUCT_ID_2', how='inner')
    df = df.drop('PRODUCT_ID_2', axis=1).rename(columns={'PRODUCT_ID': 'PRODUCT_ID_2'})

    df.to_csv(f'final_{file_name}', index=False)

In [12]:
update_interactions('interactions_upper.csv')
update_interactions('interactions_middle.csv')
update_interactions('interactions_low.csv')

In [13]:
# Updating product_desc
prods = pd.read_csv('product_desc.csv')
prods['PRODUCT_ID_2'] = prods['PRODUCT_ID']
prods['PRODUCT_ID'] = prods.index
prods = prods.drop('PRODUCT_ID_2', axis=1)

prods.to_csv('final_product_desc.csv', index=False)